In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime
import psycopg2
from sqlalchemy import create_engine
import sqlalchemy

In [43]:
def runQuery(query):
    #insert database access
   
    cursor = connection.cursor()
    if "SELECT" in query:
        table = pd.read_sql(query, con = connection)
    elif "UPDATE" in query:
        cursor.execute(query)
        print(query)
        connection.commit()
    elif "DELETE" in query:
        cursor.execute(query)
        print(query)
        connection.commit()
    else:
        return None
    if connection:
        cursor.close()
        connection.close()
        if "SELECT" in query:
            return table
        else:
            return


runQuery("SELECT * FROM u01_suzanne_mitchell.wfu_master")


,cohort,sires,dames,labanimalid,accessid,sex,rfid,dob,dow,shipmentdate,...,littersize,coatcolor,earpunch,rack,shipmentbox,housingbox,shipmentage,weanage,comments,resolution
0,C01,72539_1,72595_5,TJ008,73472_2,F,933000320045906,2018-09-24,2018-10-15,2018-10-30,...,8.0,BROWNHOOD,RB,D-F4,2.0,9999,36.0,21.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE
1,C01,73358_1,73360_1,TJ027,73477_2,F,933000320045890,2018-09-25,2018-10-16,2018-10-30,...,11.0,BROWNHOOD,LM,D-G8,4.0,32,35.0,21.0,NA,NA
2,C01,72539_1,72595_5,TJ003,73472_8,M,933000320045902,2018-09-24,2018-10-15,2018-10-30,...,8.0,ALBINO,RT,D-F2,2.0,9999,36.0,21.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE
3,C01,72794_1,72775_4,TJ020,73475_4,F,933000320045904,2018-09-25,2018-10-16,2018-10-30,...,11.0,BROWN,RB,D-G2,2.0,9999,35.0,21.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE
4,C01,72624_1,72596_4,TJ002,73471_5,F,933000320045908,2018-09-23,2018-10-15,2018-10-30,...,7.0,BLACK,LT,D-F2,2.0,9999,37.0,22.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,C07,933000320048190,933000320187377,NA,None,F,933000320125436,2020-08-09,2020-09-04,None,...,9.0,BLACK,None,None,NaN,None,NaN,26.0,NA,NA
897,C07,933000320187153,933000320187383,NA,None,F,933000320125437,2020-08-09,2020-09-04,None,...,11.0,BROWNHOOD,None,None,NaN,None,NaN,26.0,NA,NA
898,C07,933000320187217,933000320048245,NA,None,F,933000320125440,2020-08-09,2020-09-04,None,...,11.0,BROWN,None,None,NaN,None,NaN,26.0,NA,NA
899,C07,933000320187249,933000320048234,NA,None,M,933000320125443,2020-08-08,2020-09-04,None,...,10.0,BLACKHOOD,None,None,NaN,None,NaN,27.0,NA,NA


In [97]:
def qc_pedigree(data):
    data = data.sort_values(["cohort", "labanimalid"])
    # Printing number of mothers and fathers per cohort
    for cohort in np.unique(data["cohort"]):
        subset = data[data["cohort"] == cohort]
        sires = list(subset["sires"])
        dames = list(subset["dames"])
        print(cohort, "sires", "total", len(sires), "dups", len(set(sires)))
        print(cohort, "dames", "total", len(dames), "dups", len(set(dames)))
    # Printing animals listed as mother and father
    sires = list(data["sires"])
    dames = list(data["dames"])
    overlap = set(dames).intersection(set(sires))
    #print("sires", len(set(sires)), "dames", len(set(dames)), "overlap", len(overlap))
    print("listed as dame and sire", list(overlap))
    # Checking sibling numbers
    parents_dict = {}
    dob = []
    litter = []
    parents_df = pd.DataFrame()
    for index, row in data.iterrows():
        parents = (row["sires"], row["dames"])
        
        if parents in parents_dict.keys():
            parents_dict[parents].append(row["rfid"])
        else:
            dob.append(row['dob'])
            litter.append(row['littersize'])
            parents_dict[parents] = [row["rfid"]]
            parents_df = parents_df.append({'sires':row["sires"],'dames':row["dames"]},ignore_index = True)
    print("parent pairs", len(parents_dict.keys()))
    siblings = []
    for parents in parents_dict.keys():
        #print(parents, len(parents_dict[parents]))#, parents_dict[parents])
        siblings.append(len(parents_dict[parents]))
        cohorts = []
        for rfid in parents_dict[parents]:
            subset = data[data["rfid"] == rfid]
            subset.index = range(subset.shape[0])
            i_cohort = list(subset.columns).index("cohort")
            cohorts.append(subset.iloc[0, i_cohort])
        cohorts = list(np.unique(cohorts))
        if len(cohorts) != 1:
            print("parent pair across multiple cohorts found", cohorts)
    parents_df['siblings'] = siblings
    parents_df['dob'] = dob
    parents_df['litter_size'] = litter
    parents_df['removed_litter'] = parents_df['litter_size'] - parents_df['siblings']

    #print(parents_df)
    print(parents_df.litter_size.unique())
    print(data.littersize.unique())
    print(parents_df.removed_litter.unique())
    df_litter_error = parents_df[(parents_df['removed_litter'] <= 0)]
    print(df_litter_error)
    return
qc_pedigree(data)

C01 sires total 100 dups 34
C01 dames total 100 dups 34
C02 sires total 110 dups 44
C02 dames total 110 dups 44
C03 sires total 100 dups 40
C03 dames total 100 dups 40
C04 sires total 100 dups 48
C04 dames total 100 dups 48
C05 sires total 100 dups 51
C05 dames total 100 dups 51
C06 sires total 100 dups 50
C06 dames total 100 dups 50
C07 sires total 291 dups 99
C07 dames total 291 dups 99
listed as dame and sire ['nan']
parent pairs 396
parent pair across multiple cohorts found ['C01', 'C02']
parent pair across multiple cohorts found ['C01', 'C02']
parent pair across multiple cohorts found ['C01', 'C02']
parent pair across multiple cohorts found ['C01', 'C02']
parent pair across multiple cohorts found ['C01', 'C02']
parent pair across multiple cohorts found ['C02', 'C03']
parent pair across multiple cohorts found ['C02', 'C03']
parent pair across multiple cohorts found ['C04', 'C06']
parent pair across multiple cohorts found ['C04', 'C06']
parent pair across multiple cohorts found ['C0

In [30]:
#Split data based on cohorts
#each cohort has their own df named after their cohort. Ex. C07
cohorts = data.cohort.unique()
for i in cohorts:
    a = data[data.cohort == i]
    globals()[f"{i}"] = a
cohorts    

array(['C01', 'C04', 'C05', 'C02', 'C03', 'C06', 'C07'], dtype=object)

In [40]:
#Check that the dob for each cohort is within a certain range
#right now just outputs the cohort with the day span
dob_range = {}
for i in cohorts:
    globals()[f"dates_{i}"] = globals()[f"{i}"].dob.unique()
    oldest = min(globals()[f"dates_{i}"])
    youngest = max(globals()[f"dates_{i}"])
    difference = youngest - oldest
    dob_range[i] = difference
dob_range

{'C01': datetime.timedelta(days=11),
 'C04': datetime.timedelta(days=13),
 'C05': datetime.timedelta(days=13),
 'C02': datetime.timedelta(days=13),
 'C03': datetime.timedelta(days=13),
 'C06': datetime.timedelta(days=19),
 'C07': datetime.timedelta(days=71)}

In [18]:
#Check if there are any matching ids between sires and dames
unique_sires = data.sires.unique()
unique_dames = data.dames.unique()
print([x for x in unique_sires if x in unique_dames])


['nan']


In [46]:
#Check namiing of coat colors
unique_coats = data.coatcolor.unique()
unique_coats


array(['BROWNHOOD', 'ALBINO', 'BROWN', 'BLACK', 'BLACKHOOD'], dtype=object)